# Introduction to Tensor Network Contractions using `np.einsum`

In this notebook, you will learn how to use NumPy's `np.einsum` function to perform **tensor network contractions**. We'll cover the basics of `np.einsum`, how to use it for complex contractions, and how to compute the **optimal contraction path** for improved runtime performance.

## Table of Contents
1. What is `np.einsum`?
2. Basic Usage of `np.einsum`
3. Tensor Network Contractions with `np.einsum`
4. Computing the Optimal Path with `np.einsum_path`
5. The `optimize` parameter
6. Opt_einsum
7. Exercises


## 1. What is `np.einsum`?

The function `np.einsum` (Einstein Summation) is a powerful tool in NumPy that allows for complex array operations, including:

- Summation over multiple axes
- Transposing arrays
- Inner/Outer products
- Generic Matrix and tensor contractions

The syntax of `np.einsum` uses Einstein summation notation to specify how tensors should be contracted or manipulated. This concise notation eliminates the need for explicit loops or intermediate operations.

## 2. Basic Usage of `np.einsum`

Let's start with a few simple examples to understand how `np.einsum` works.

### Example 1: Matrix Multiplication

![](../img/matmul.png)

Matrix multiplication between two 2D arrays can be performed as follows:

In [42]:
# Import necessary library
import numpy as np

# Define two matrices
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

# Perform matrix multiplication using einsum
C = np.einsum('ij,jk->ik', A, B)
print('Result of matrix multiplication using einsum:\n', C)

Result of matrix multiplication using einsum:
 [[19 22]
 [43 50]]


The string `'ij,jk->ik'` tells `np.einsum` how to contract the tensors:
- The first matrix `A` is represented by the indices `ij`.
- The second matrix `B` is represented by the indices `jk`.
- The resulting matrix `C` is indexed by `ik`, meaning that the index `j` is summed over (contracted).

This corresponds to the traditional definition of matrix multiplication.

### Example 2: Element-wise Summation

![](../img/trace.png)

You can also use `np.einsum` for element-wise operations. For example, summing over all elements of a matrix can be done as follows:

In [43]:
# Define a matrix
A = np.array([[1, 2], [3, 4]])

# Sum all elements using einsum
sum_A = np.einsum('ij->', A)
print('Sum of all elements in A:', sum_A)

Sum of all elements in A: 10


The notation `'ij->'` indicates that the indices `i` and `j` should be summed over, resulting in a scalar. 

## 3. Tensor Network Contractions with `np.einsum`

Tensor networks consist of nodes (tensors) connected by edges, which represent the contraction of indices between tensors. `np.einsum` can efficiently handle these contractions.

Let's perform a more complex contraction between three tensors.

![](../img/contract3.png)

In [44]:
# Index dimensions
Di = 30
Dj = 50
Dk = 20
Dl = 50
Dm = 20
# Define three tensors
A = np.random.rand(Di, Dj, Dk) # A_ijk
B = np.random.rand(Dj, Dl) # B_jl
C = np.random.rand(Dk, Dm) # C_km

# Perform a contraction over multiple tensors
D = np.einsum('ijk,jl,km->ilm', A, B, C) # D_ilm
print('Result of tensor network contraction:\n', result)

Result of tensor network contraction:
 [[2.95215913 2.90979933 1.83302395 1.4888609  1.56921193]
 [3.50377242 3.41674896 2.23219287 1.63371673 1.81712126]]


In this case, we are contracting the following indices:
- `A[i,j,k]` is contracted with `B[j,l]` over index `j`.
- `A[i,j,k]` is contracted with `C[k,m]` over index `k`.

This is a typical tensor network contraction performed with `np.einsum`.

## 4. Computing the Optimal Path with `np.einsum_path`

When contracting large tensor networks, the order of contraction can significantly affect performance. **`np.einsum_path`** can be used to compute the optimal contraction path that minimizes both computational cost and memory usage.

The function `np.einsum_path` outputs optimal contraction order along with information including the scaling and the size of the largest intermediate tensors. 

In [45]:
# Compute the optimal path for contracting three tensors
opt_path, path_info = np.einsum_path('ijk,jl,km->ilm', A, B, C, optimize='optimal')
print('Optimal Contraction Path:', opt_path)
print('\nDetailed Path Information:')
print(path_info)

Optimal Contraction Path: ['einsum_path', (0, 1), (0, 1)]

Detailed Path Information:
  Complete contraction:  ijk,jl,km->ilm
         Naive scaling:  5
     Optimized scaling:  4
      Naive FLOP count:  9.000e+07
  Optimized FLOP count:  4.200e+06
   Theoretical speedup:  21.429
  Largest intermediate:  3.000e+04 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
   4                 jl,ijk->kil                              km,kil->ilm
   4                 kil,km->ilm                                 ilm->ilm


### Explanation of Output:
1. **Optimal Contraction Path**: This lists the optimal order of contraction, minimizing computational complexity.
2. **Path Information**: This includes detailed information about each contraction step, the scaling and memory usage.

This optimal path computation can significantly improve runtime when contracting large tensor networks, especially when the tensors are high-dimensional or very large.

### Breakdown of Path Information

- **Naive Scaling**: The naive runtime scaling: this will just be the number of indices involved in the contraction.

- **Optimized Scaling**: The runtime scaling with the optimized contraction order.

- **Largest intermediate**: This tells you the size (number of elements) of the largest intermediate tensor that is computed during the contraction. Large intermediate tensors can be a source of inefficiency, as they require more memory and may lead to slower performance.

## 5. The `optimize` parameter

The `optimize` parameter in `np.einsum` controls how the contraction path is computed. Different settings can lead to different execution times, particularly for large tensor networks.

We can compare contraction times for different values of `optimize`:

- `optimize=False`: No optimization, performs contractions as specified.
- `optimize='greedy'`: Greedy optimization to reduce contraction cost, often faster but not always optimal.
- `optimize='optimal'`: Uses dynamic programming to find the optimal contraction path. This can be more expensive to compute but often results in the fastest runtime for large tensors.
- `optimize=path`: We manually set the contraction path. We can use the output of `np.einsum_path` to set the optimal path. This should be the fastest option as the optimal path has already been computed in advance.

We'll compare these values by timing a tensor contraction using each option.

In [46]:
%%timeit
# 1. Optimize=False
result_no_opt = np.einsum('ijk,jl,km->ilm', A, B, C, optimize=False)

29.1 ms ± 693 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
%%timeit
# 2. Optimize='greedy'
result_greedy_opt = np.einsum('ijk,jl,km->ilm', A, B, C, optimize='greedy')

190 μs ± 12.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [48]:
%%timeit
# 3. Optimize='optimal'
result_optimal_opt = np.einsum('ijk,jl,km->ilm', A, B, C, optimize='optimal')

189 μs ± 20.8 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [49]:
%%timeit
# 4. Optimize=opt_path
result_optimal_opt = np.einsum('ijk,jl,km->ilm', A, B, C, optimize=opt_path)

163 μs ± 4.67 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


We can see that setting the precomputed optimal path is the fastest. The slowest is `optimize=False` because this uses the sub-optimal path (scaling 5). Using the greedy algorithm is ever so slighly faster than 'optimal' since the path computed is the same but computing the path takes less time.

## 6. Opt_einsum

We have chosen to explore tensor contraction using NumPy's einsum function, however it is important to point out that is not the most performant option out there. For a faster and more feature rich function check out opt_einsum:

https://optimized-einsum.readthedocs.io/en/stable/index.html

## 7. Exercises

Now that you understand the basics of using `np.einsum` and how to compute the optimal contraction path, try the following exercises to test your comprehension.

### Exercise 1: Matrix Transposition and Multiplication
1. Create two matrices `A` of shape `(3, 2)` and `B` of shape `(3, )` filled with random integers.
2. Use `np.einsum` to compute the transpose of matrix `A` and then multiply it by matrix `B`.
3. Verify the result using standard matrix operations.

### Exercise 2: Outer Product
1. Create two 1D arrays `x` and `y` of length 4.
2. Use `np.einsum` to compute the outer product of `x` and `y`.
3. Verify the result using `np.outer` and compare.

### Exercise 3: Tensor Contractions

For tensor networks 1, 2 and 3.

![](../img/contract_ex.png)
a. What is the order of the resulting tensor? Eg, scalar, matrix, ...

b. What is the optimal runtime scaling of the contraction

c. Create arrays corresponding to the tensors filled with random numbers.

d. Use `np.einsum` to contract the network.

e. Verify your answer to part a. by inspecting the result of `np.einsum`

f. Verfiy your answer to part b. by inspecting the result of `np.einsum_path`